In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

In [2]:

FILE_PATH = "/kaggle/input/modified-critical-role-scripts/script.txt"

IS_DEBUG = False
SAVE_MODEL = False

text = open(FILE_PATH, "rb").read().decode(encoding="utf-8")

vocab = sorted(set(text))
vocab_size = len(vocab)

ids_from_chars = preprocessing.StringLookup(vocabulary=list(vocab), mask_token=None)

chars_from_ids = preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None
)

def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)


def split_input_sequence(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text


all_ids = ids_from_chars(tf.strings.unicode_split(text, "UTF-8"))
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)


seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

dataset = sequences.map(split_input_sequence)

BATCH_SIZE = 64
BUFFER_SIZE = 10000
EPOCHS = 40

if IS_DEBUG:
    EPOCHS = 1

embedding_dim = 256
rnn_units = 2048

dataset = (
    dataset.shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

2022-08-01 22:02:02.442284: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [3]:
class CriticalRoleModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(
            rnn_units, return_sequences=True, return_state=True
        )
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

model = CriticalRoleModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
)

loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])


In [4]:
history = model.fit(dataset, epochs=EPOCHS)

Epoch 1/40


2022-08-01 22:02:08.909299: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


35/35 [==============================] - 276s 8s/step - loss: 3.9119 - accuracy: 0.1862
Epoch 2/40
35/35 [==============================] - 267s 8s/step - loss: 2.4963 - accuracy: 0.3425
Epoch 3/40
35/35 [==============================] - 267s 8s/step - loss: 2.1005 - accuracy: 0.4124
Epoch 4/40
35/35 [==============================] - 266s 8s/step - loss: 1.9349 - accuracy: 0.4447
Epoch 5/40
35/35 [==============================] - 267s 8s/step - loss: 1.8217 - accuracy: 0.4692
Epoch 6/40
35/35 [==============================] - 265s 8s/step - loss: 1.7154 - accuracy: 0.4969
Epoch 7/40
35/35 [==============================] - 264s 8s/step - loss: 1.6209 - accuracy: 0.5215
Epoch 8/40
35/35 [==============================] - 265s 8s/step - loss: 1.5334 - accuracy: 0.5423
Epoch 9/40
35/35 [==============================] - 264s 8s/step - loss: 1.4497 - accuracy: 0.5652
Epoch 10/40
35/35 [==============================] - 266s 8s/step - loss: 1.3754 - accuracy: 0.5846
Epoch 11/40
35/35 [=

In [5]:
class OneStepModel(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars):
        super().__init__()
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        skip_ids = self.ids_from_chars(["[UNK]"])[:, None]

        sparse_mask = tf.SparseTensor(
            values=[-float("inf")] * len(skip_ids),
            indices=skip_ids,
            dense_shape=[len(ids_from_chars.get_vocabulary())],
        )
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        input_chars = tf.strings.unicode_split(inputs, "UTF-8")
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(
            inputs=input_ids, states=states, return_state=True
        )
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [6]:
one_step_model = OneStepModel(model, chars_from_ids, ids_from_chars)

states = None

next_char = tf.constant(["MATT:"])
result = [next_char]

length = 150

for n in range(length):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)


# file = open("generated.txt", "w")

# file.write(result[0].numpy().decode("utf-8"))

# file.close()

print("Finished!")

print(result[0].numpy().decode("utf-8"))

print( "\n\n" + "_" * 80 + "\n\n" )

Finished!
MATT: That ends your turn?

MARISHA: Yep.

MATT: All righty, where do you want it to go?

LAURA: Wait. Wouldn't it be funny if that was
Marius?

M


________________________________________________________________________________


